In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
main_folder = "/content/gdrive/My Drive/Ext-val/"

In [ ]:
conn_measures = ['PAC', 'SCR-(1,4)', 'SCI-(1,4)', 'PLV-(1,4)', 'PLI-(1,4)', 'CC-(1,4)', 'SCR-(4,8)', 'SCI-(4,8)', 'PLV-(4,8)', 'PLI-(4,8)', 'CC-(4,8)', 'SCR-(8,13)', 'SCI-(8,13)', 'PLV-(8,13)', 'PLI-(8,13)', 'CC-(8,13)', 'SCR-(13,30)', 'SCI-(13,30)', 'PLV-(13,30)', 'PLI-(13,30)', 'CC-(13,30)', 'SCR-(30,70)', 'SCI-(30,70)', 'PLV-(30,70)', 'PLI-(30,70)', 'CC-(30,70)', 'SCR-(70,150)', 'SCI-(70,150)', 'PLV-(70,150)', 'PLI-(70,150)', 'CC-(70,150)']

In [ ]:
import numpy as np

def to_labels(pos_probs, threshold):
    # function to map all values >=threshold to 1 and all values <threshold to 0

	return list((pos_probs >= threshold).astype('int'))


def moving_thresh_auc(predictive_measure=[], outcome=[], n_good=17, n_bad=7, moving_step=0.00001):
    # returns AUC, best threshold, true negatives and true positives at the best threshold

    thresholds = np.arange(0, np.max(predictive_measure), moving_step)

    g = np.array([pm for i,pm in enumerate(predictive_measure) if outcome[i]=="good"])
    b = np.array([pm for i,pm in enumerate(predictive_measure) if outcome[i]=="bad"])

    A, A_top = 0, 0
    T = 0
    tp_top, tn_top = 0, 0
    step = 0
    for t in thresholds:    
        g_l, b_l = to_labels(g, t), to_labels(b, t)
        tp = sum(g_l)/n_good 
        tn = b_l.count(0)/n_bad
        A = (tp + tn)/2
        if A>A_top: 
            step=0
            A_top=A
            T=t
            tn_top,tp_top=tn,tp
        elif A==A_top: step+=moving_step

    return (A_top, T, tn_top, tp_top)

The game outcome data was saved in a separate Excel file for each subject and for thresholds of 2, 3 or 4 sigmas. To predict surgical outcomes, the data needs to be fused into a single dataframe.  

In [ ]:
import os
import pandas as pd

# Path to the directory containing Excel files
xlsx_dir = main_folder + 'data_per_sigma/'

# Initialize empty dataframes for each type
df_0sigma = pd.DataFrame()
df_1sigma = pd.DataFrame()
df_2sigma = pd.DataFrame()
df_3sigma = pd.DataFrame()
df_4sigma = pd.DataFrame()

# Iterate through files in the directory
for filename in os.listdir(xlsx_dir):
    if filename.endswith('.xlsx'):
        # Extract type and subject ID from filename
        parts = filename.split('_')
        file_type = parts[-2]
        subject_id = parts[-1].split('.')[0]

        # Read Excel file into a dataframe
        file_path = os.path.join(xlsx_dir, filename)
        df = pd.read_excel(file_path)

        # Concatenate dataframe to the corresponding type dataframe
        if file_type == '0sigma':
            df_0sigma = pd.concat([df_1sigma, df], ignore_index=True)        
        elif file_type == '1sigma':
            df_1sigma = pd.concat([df_1sigma, df], ignore_index=True)
        elif file_type == '2sigma':
            df_2sigma = pd.concat([df_2sigma, df], ignore_index=True)
        elif file_type == '3sigma':
            df_3sigma = pd.concat([df_3sigma, df], ignore_index=True)
        elif file_type == '4sigma':
            df_4sigma = pd.concat([df_4sigma, df], ignore_index=True)



In [ ]:
# Save dataframes to Excel files
df_0sigma.to_excel(os.path.join(xlsx_dir,'surgical_outcome_data_0sigma.xlsx'), index=False)
df_1sigma.to_excel(os.path.join(xlsx_dir,'surgical_outcome_data_1sigma.xlsx'), index=False)
df_2sigma.to_excel(os.path.join(xlsx_dir,'surgical_outcome_data_2sigma.xlsx'), index=False)
df_3sigma.to_excel(os.path.join(xlsx_dir,'surgical_outcome_data_3sigma.xlsx'), index=False)
df_4sigma.to_excel(os.path.join(xlsx_dir,'surgical_outcome_data_4sigma.xlsx'), index=False)


The code below checks if there are any zero Mean_overlap_ratio in the data. If yes, it fetches the value from the same index in the previous sigma file (sigma-1) and continues with the analysis. If a zero is found in row `zero_row_index` in the current sigma file, it replaces the corresponding value with the one from the previous sigma file.


In [ ]:
# !pip install statannot

import seaborn as sns
from statannot import add_stat_annotation
from scipy.stats import shapiro, mannwhitneyu, ttest_ind
from itertools import combinations

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
import matplotlib.pyplot as plt

In [ ]:
sigma = 4
while sigma > 0:
    load_data = pd.read_excel(xlsx_dir + f"surgical_outcome_data_{sigma}sigma.xlsx", engine='openpyxl')

    # Check if "Mean_overlap_ratio" contains any zeros
    zero_row_indices = load_data.index[load_data["Mean_overlap_ratio"] == 0].tolist()

    if zero_row_indices:
        print(f"Zeros found in rows {zero_row_indices} for sigma={sigma}")
        if sigma == 0:
            print("Skipping iteration for sigma=1 with zeros")
            break

        sigma -= 1
        prev_load_data = pd.read_excel(xlsx_dir + f"surgical_outcome_data_{sigma}sigma.xlsx", engine='openpyxl')

        for zero_row_index in zero_row_indices:
            zero_value = prev_load_data.loc[zero_row_index, "Mean_overlap_ratio"]
            load_data.loc[zero_row_index, "Mean_overlap_ratio"] = zero_value
    else:
        break  # No zeros found, exit the loop



In [ ]:
CM, Sigma, Summary_stat, Gauss, Pvalue_Shapiro, Test, Pvalue, MAUC, T, TN, TP = [], [], [], [], [], [], [], [], [], [], []

summary_stat_label = ["mm"]
n_good=17
n_bad=7
for cm in conn_measures:
    for summary_stat in summary_stat_label:
        data = load_data.groupby("CM").get_group(cm).groupby("Strategy").get_group(summary_stat)

        if 16 not in list(data.Subject): n_bad=6
        else: n_bad=7

        x_plot, y_plot = list(data.Mean_overlap_ratio), ["good" if val == 1 else "bad" for val in data["Outcome"]]

        mauc = moving_thresh_auc(x_plot, y_plot, moving_step=0.0001, n_good=n_good, n_bad=n_bad)

        group1 = [x for i, x in enumerate(x_plot) if y_plot[i] == "good"]
        group0 = [x for i, x in enumerate(x_plot) if y_plot[i] == "bad"]

        gaussian, stest = False, ''
        stat1, p1 = shapiro(group1)
        stat0, p0 = shapiro(group0)

        pvalue = 1
        if p1 > 0.05 or p0 > 0.05:
            _, pvalue = mannwhitneyu(group1, group0, alternative='two-sided')
            stest = 'Mann-Whitney'
            gaussian = True
        elif p1 <= 0.05 and p0 <= 0.05:
            _, pvalue = ttest_ind(group1, group0, equal_var=False)
            stest = 't-test_welch'

        g = sns.catplot(data=data, x='Outcome', y='Mean_overlap_ratio', order=[1, 0], kind="box", hue="Outcome",
                        palette=["#FFA7A0", "#ABEAC9"],
                        height=4, aspect=.7)
        g.map_dataframe(sns.stripplot, x='Outcome', y='Mean_overlap_ratio', order=[1, 0], hue="Outcome",
                        palette=["#404040", "#404040"],
                        alpha=0.6, dodge=True)

        # p_annot = add_stat_annotation(g.axes[0][0], data=data, x=x_plot, y=y_plot, order=["good", "bad"],
        #                               box_pairs=[("good", "bad")],
        #                               test=stest, text_format='star', loc='outside', verbose=0)

        # pvalue_statannot = p_annot[1][0].__dict__["pval"]
        if pvalue <= 0.05:
            # print("P-value computed by statannot =", pvalue_statannot)
            print(stest)
            plt.title(f"Measure = {cm}; summary_stat = {summary_stat}; p-value = {pvalue}; AUC = {mauc[0]}", pad=50)
            plt.show()

        plt.close('all')

        Sigma.append(sigma)
        CM.append(cm)
        Summary_stat.append(summary_stat)
        Gauss.append(gaussian)
        Pvalue_Shapiro.append((p0, p1))
        Test.append(stest)
        Pvalue.append(pvalue)
        MAUC.append(mauc[0])
        T.append(mauc[1])
        TN.append(mauc[2])
        TP.append(mauc[3])

outcome_pred = pd.DataFrame({"CM": CM, "Summary_stat": summary_stat,
                          "Gauss": Gauss, "Pvalue-Shapiro": Pvalue_Shapiro,
                          "Test": Test, "Pvalue": Pvalue,
                          "MAUC": MAUC, "T": T, "TN": TN, "TP": TP})
outcome_pred.to_excel(main_folder + f"surgical_outcome_prediction_game_vKarla_sigma_recursive.xlsx")
pass;

Combinations of 2 connectivity measures.

In [ ]:
CM, Summary_stat, Gauss, Pvalue_Shapiro, Test, Pvalue, MAUC, T, TN, TP = [], [], [], [], [], [], [], [], [], []

summary_stat_label = ["mm"]
n_good=17
n_bad=7
for cm in combinations(conn_measures, 2): # combinations of scores based on different connectivity measures
    cm1,cm2 = cm[0],cm[1]
    # print(cm1,cm2)

    for summary_stat in summary_stat_label:

        data1 = load_data.groupby("CM").get_group(cm1).groupby("Strategy").get_group(summary_stat)
        data2 = load_data.groupby("CM").get_group(cm2).groupby("Strategy").get_group(summary_stat)

        if 16 not in list(data1.Subject) and 16 in list(data2.Subject): data2 = data2[data2['Subject'] != 16]; n_bad=6
        elif 16 not in list(data2.Subject) and 16 in list(data1.Subject): data1 = data1[data1['Subject'] != 16]; n_bad=6
        else: n_bad=7

        # the combination is made as a mean value between two scores
        x_plot = [np.mean([val, list(data2.Mean_overlap_ratio)[i]]) for i,val in enumerate(data1.Mean_overlap_ratio)]
        y_plot = ["good" if val==1 else "bad" for val in data1["Outcome"]]
        
        data = pd.DataFrame({"Mean_overlap_ratio":x_plot,
                                "Outcome":y_plot})

        # calculate moving threshold-based AUC
        mauc = moving_thresh_auc(x_plot, y_plot, moving_step=0.0001, n_good=n_good, n_bad=n_bad)

        group1 = [x for i,x in enumerate(x_plot) if y_plot[i]=="good"]
        group0 = [x for i,x in enumerate(x_plot) if y_plot[i]=="bad"]

        gaussian, stest = False, ''
        stat1, p1 = shapiro(group1)
        stat0, p0 = shapiro(group0)

        pvalue = 1
        if p1 > 0.05 or p0 > 0.05: _, pvalue = mannwhitneyu(group1, group0, alternative='two-sided'); stest = 'Mann-Whitney'; gaussian = True
        elif p1 <= 0.05 and p0 <= 0.05: _, pvalue = ttest_ind(group1, group0, equal_var = False); stest = 't-test_welch'

        g = sns.catplot(data=data, x='Outcome', y='Mean_overlap_ratio', order=["good","bad"], kind="box", hue="Outcome",
                        palette=["#ABEAC9", "#FFA7A0"],
                        height=4, aspect=.7);
        g.map_dataframe(sns.stripplot, x='Outcome', y='Mean_overlap_ratio', order=["good","bad"], hue="Outcome",
                        palette=["#404040","#404040"],
                        alpha=0.6, dodge=True);

        # p_annot = add_stat_annotation(g.axes[0][0], data=data, x=x_plot, y=y_plot, order=["good","bad"],
        #                 box_pairs=[("good","bad")],
        #                 test=stest, text_format='star', loc='outside', verbose=0);

        # pvalue_statannot = p_annot[1][0].__dict__["pval"]
        if pvalue<=0.05:
            # print("P-value computed by statannot =", pvalue_statannot)
            print(stest)
            print(summary_stat)
            plt.title(f"Measures = {(cm1, cm2)}; p-value = {pvalue:.4f}; AUC = {mauc[0]:.2f}", pad=50)
            plt.show()

        plt.close('all')

        CM.append((cm1,cm2))
        Summary_stat.append(summary_stat)
        Gauss.append(gaussian)
        Pvalue_Shapiro.append((p0, p1))
        Test.append(stest)
        Pvalue.append(pvalue)
        MAUC.append(mauc[0])
        T.append(mauc[1])
        TN.append(mauc[2])
        TP.append(mauc[3])

outcome_pred = pd.DataFrame({"CM": CM, "Summary_stat": summary_stat,
                          "Gauss": Gauss, "Pvalue-Shapiro": Pvalue_Shapiro,
                          "Test": Test, "Pvalue": Pvalue,
                          "MAUC": MAUC, "T": T, "TN": TN, "TP": TP})

outcome_pred.to_excel(main_folder + f"surgical_outcome_prediction_game_vKarla_sigma_recursive_comb2.xlsx")
pass;

Combinations of 3 connectivity measures.

In [ ]:
CM, Summary_stat, Gauss, Pvalue_Shapiro, Test, Pvalue, MAUC, T, TN, TP = [], [], [], [], [], [], [], [], [], []

summary_stat_label = ["mm"]

for cm in combinations(conn_measures, 3): # combinations of scores based on different connectivity measures
    cm1,cm2,cm3 = cm[0],cm[1],cm[2]
    # print(cm1,cm2,cm3)

    for summary_stat in summary_stat_label:
        n_good, n_bad = 17, 7
        data1 = load_data.groupby("CM").get_group(cm1).groupby("Strategy").get_group(summary_stat)
        data2 = load_data.groupby("CM").get_group(cm2).groupby("Strategy").get_group(summary_stat)
        data3 = load_data.groupby("CM").get_group(cm3).groupby("Strategy").get_group(summary_stat)

        if 16 not in list(data1.Subject) or 16 not in list(data2.Subject) or 16 not in list(data3.Subject):
            data1 = data1[data1['Subject'] != 16]
            data2 = data2[data2['Subject'] != 16]
            data3 = data3[data3['Subject'] != 16]            
            n_bad=6

        # the combination is made as a mean value between two scores
        x_plot = [np.mean([val, list(data2.Mean_overlap_ratio)[i], list(data3.Mean_overlap_ratio)[i]]) for i,val in enumerate(data1.Mean_overlap_ratio)]
        y_plot = ["good" if val==1 else "bad" for val in data1["Outcome"]]

        data = pd.DataFrame({"Mean_overlap_ratio":x_plot,
                                "Outcome":y_plot})

        # calculate moving threshold-based AUC
        mauc = moving_thresh_auc(x_plot, y_plot, moving_step=0.0001, n_good=n_good, n_bad=n_bad)

        group1 = [x for i,x in enumerate(x_plot) if y_plot[i]=="good"]
        group0 = [x for i,x in enumerate(x_plot) if y_plot[i]=="bad"]

        gaussian, stest = False, ''
        stat1, p1 = shapiro(group1)
        stat0, p0 = shapiro(group0)

        pvalue = 1
        if p1 > 0.05 or p0 > 0.05: _, pvalue = mannwhitneyu(group1, group0, alternative='two-sided'); stest = 'Mann-Whitney'; gaussian = True
        elif p1 <= 0.05 and p0 <= 0.05: _, pvalue = ttest_ind(group1, group0, equal_var = False); stest = 't-test_welch'

        g = sns.catplot(data=data, x='Outcome', y='Mean_overlap_ratio', order=["good","bad"], kind="box", hue="Outcome",
                        palette=["#ABEAC9", "#FFA7A0"],
                        height=4, aspect=.7);
        g.map_dataframe(sns.stripplot, x='Outcome', y='Mean_overlap_ratio', order=["good","bad"], hue="Outcome",
                        palette=["#404040","#404040"],
                        alpha=0.6, dodge=True);

        # p_annot = add_stat_annotation(g.axes[0][0], data=data, x=x_plot, y=y_plot, order=["good","bad"],
        #                 box_pairs=[("good","bad")],
        #                 test=stest, text_format='star', loc='outside', verbose=0);

        # pvalue_statannot = p_annot[1][0].__dict__["pval"]
        if pvalue<=0.05:
            # print("P-value computed by statannot =", pvalue_statannot)
            print(stest)
            print(summary_stat)
            plt.title(f"Measures = {(cm1, cm2, cm3)}; p-value = {pvalue:.4f}; AUC = {mauc[0]:.2f}", pad=50)
            plt.show()

        plt.close('all')

        CM.append((cm1,cm2,cm3))
        Summary_stat.append(summary_stat)
        Gauss.append(gaussian)
        Pvalue_Shapiro.append((p0, p1))
        Test.append(stest)
        Pvalue.append(pvalue)
        MAUC.append(mauc[0])
        T.append(mauc[1])
        TN.append(mauc[2])
        TP.append(mauc[3])

outcome_pred = pd.DataFrame({"CM": CM, "Summary_stat": summary_stat,
                          "Gauss": Gauss, "Pvalue-Shapiro": Pvalue_Shapiro,
                          "Test": Test, "Pvalue": Pvalue,
                          "MAUC": MAUC, "T": T, "TN": TN, "TP": TP})

outcome_pred.to_excel(main_folder + f"surgical_outcome_prediction_game_vKarla_sigma_recursive_comb3.xlsx")
pass;